In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import sklearn.metrics

In [2]:
class SlimAlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1,32,kernel_size = 3, stride =1),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(kernel_size =3, stride = 2),
            nn.Conv2d(32,64,kernel_size = 3),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(kernel_size =3, stride = 2),
            nn.Conv2d(64,128,kernel_size = 3, padding =1),
            nn.ReLU(inplace = True),
            nn.Conv2d(128,256,kernel_size = 3, padding =1),
            nn.ReLU(inplace = True),
            nn.Conv2d(256,128,kernel_size = 3, padding =1),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(kernel_size =3, stride = 2),
            )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(128,1024),
            nn.ReLU(inplace= True),
            nn.Dropout(),
            nn.Linear(1024,1024),
            nn.ReLU(inplace= True),
            nn.Linear(1024,num_classes)
            )
    def forward(self,x):
        x = self.features(x)
        x = x.flatten(start_dim=1)
        x = self.classifier(x)
        return x 

In [3]:
mnist = torchvision.datasets.MNIST('./var',download = True)
transform = transforms.Compose([transforms.ToTensor()])
train = torchvision.datasets.MNIST('./var',train =True, transform = transform)
trainloader = torch.utils.data.DataLoader(train,batch_size =256,shuffle=True)
test = torchvision.datasets.MNIST('./var',train =False, transform = transform)
testloader = torch.utils.data.DataLoader(test,batch_size =len(test),shuffle=True)

Extracting ./var/MNIST/raw/train-images-idx3-ubyte.gz to ./var/MNIST/raw


Extracting ./var/MNIST/raw/train-labels-idx1-ubyte.gz to ./var/MNIST/raw


Extracting ./var/MNIST/raw/t10k-images-idx3-ubyte.gz to ./var/MNIST/raw


Extracting ./var/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./var/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [17]:
net = SlimAlexNet(num_classes=10)
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters())
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
net.to(device)

best_loss = float('inf')
best_epoch = 1
best_acc = 0
for epoch in range(50):
    totoal_loss = 0
    net.train()
    for inputs,outputs in trainloader:
        inputs = inputs.to(device)
        outputs = outputs.to(device)
        optimizer.zero_grad()
        results = net(inputs)
        loss = loss_function(results,outputs)
        totoal_loss += loss.item()
        loss.backward()
        optimizer.step()

        #if loss < best_loss:
            #best_loss = loss
           # best_epoch = epoch
            
            #torch.save(net.state_dict(), 'best_model.pt')

    print (f'Epoch: {epoch+1:02} |Loss:{totoal_loss/len(trainloader)}')

    net.eval()
    with torch.no_grad():
        for inputs,actual in testloader:
            inputs = inputs.to(device)  
            results = net(inputs).argmax(dim=1).to('cpu').numpy()
            accuracy = sklearn.metrics.accuracy_score(actual,results)
    
        if accuracy >= best_acc:
            best_acc = accuracy
            best_epoch = epoch
            torch.save(net.state_dict(), 'best_model.pt')
            
        print("curr_val_accuracy:",accuracy, "epoch:", epoch+1, "best_acc",best_acc,"best_epoch",best_epoch)



Epoch: 01 |Loss:0.6746256555331515
val_accuracy: 0.9747 best_epoch: 1
Epoch: 02 |Loss:0.12380847521918885
val_accuracy: 0.9811 best_epoch: 2
Epoch: 03 |Loss:0.086896698398793
val_accuracy: 0.9865 best_epoch: 3
Epoch: 04 |Loss:0.06547744592374309
val_accuracy: 0.9856 best_epoch: 4
Epoch: 05 |Loss:0.052293259630653455
val_accuracy: 0.9874 best_epoch: 5
Epoch: 06 |Loss:0.04439389620451851
val_accuracy: 0.9909 best_epoch: 6
Epoch: 07 |Loss:0.037324308604002
val_accuracy: 0.9884 best_epoch: 7
Epoch: 08 |Loss:0.03600852533065258
val_accuracy: 0.9899 best_epoch: 8
Epoch: 09 |Loss:0.03141789478348925
val_accuracy: 0.9892 best_epoch: 9
Epoch: 10 |Loss:0.029547412832208138
val_accuracy: 0.9896 best_epoch: 10
Epoch: 11 |Loss:0.02560586930906519
val_accuracy: 0.9904 best_epoch: 11
Epoch: 12 |Loss:0.02235452180450901
val_accuracy: 0.9895 best_epoch: 12
Epoch: 13 |Loss:0.023446717089478007
val_accuracy: 0.9917 best_epoch: 13
Epoch: 14 |Loss:0.021682655010768707
val_accuracy: 0.9895 best_epoch: 14
Ep

In [18]:
net = SlimAlexNet(num_classes=10)
net.to(device)
net.load_state_dict(torch.load('best_model.pt'))

for inputs,actual in testloader:
    inputs = inputs.to(device)
    results = net(inputs).argmax(dim=1).to('cpu').numpy()
    accuracy = sklearn.metrics.accuracy_score(actual,results)
    print(accuracy)
print(sklearn.metrics.classification_report(actual,results))

0.9918
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       980
           1       1.00      1.00      1.00      1135
           2       1.00      0.99      0.99      1032
           3       1.00      0.99      0.99      1010
           4       0.99      1.00      0.99       982
           5       0.98      0.99      0.98       892
           6       0.99      0.99      0.99       958
           7       0.98      1.00      0.99      1028
           8       0.99      0.99      0.99       974
           9       0.99      0.99      0.99      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000



In [19]:
net = SlimAlexNet(num_classes=10)
net.to(device)
net.load_state_dict(torch.load('best_model.pt'))

from torchvision.utils import save_image
import torch
import torchvision
#save_image([0][0][0], 'img1.png')

from PIL import Image
im = Image.open("im3.png").convert('L')
im = im.resize((28,28),Image.NEAREST)
actual =  3
actual = np.array(actual)
inputs = transform(im)
inputs = inputs.to(device)
#print(inputs.size())
results = net(inputs.unsqueeze(0)).argmax(dim=1).to('cpu').numpy()
#print(type(results),type(actual),actual)
print("result:", results[0])
print("actual:", actual)
#accuracy = sklearn.metrics.accuracy_score(actual,results)
#print(accuracy)
#print(sklearn.metrics.classification_report(actual,results))

result: 8
actual: 3


In [ ]:
inputs[0,0].size()

torch.Size([28])

In [ ]:
from torchvision.utils import save_image
import torch
import torchvision
save_image(inputs[0,0], 'img1.png')

In [ ]:
from PIL import Image
im = Image.open("img1.png")



In [ ]:
im.size

(28, 28)

In [ ]:
inputs = transform(im)

In [ ]:
inputs.size()

torch.Size([3, 28, 28])

In [ ]:
test[0][0][0].size()

torch.Size([28, 28])